In [3]:
# imports
%load_ext autoreload
%autoreload 2
from dataset_wrapper import find_and_load_datasets, sample_n_from_each_dataset
from classifier_wrapper import SKLearnClassifierWrapper
from preprocessing_wrapper import PreprocessingWrapper
from logger import Logger
from features import FeatureExtraction



from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# other models, preprocessing, scalers etc.

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# paths, constants
root = "../dataset-private/" # path to the datasets
validation_split=0.1

Current limits: soft=-1, hard=-1
Failed to set resource limit: New soft limit exceeds the hard limit.


In [ ]:
experiment_name = "testing1"
model_save_path = f"./models/model_{experiment_name}.pkl"

## Loading, init

In [3]:
loaders = find_and_load_datasets(root)
print("Found datasets:", list(loaders.keys()))

results = sample_n_from_each_dataset(loaders,n=3)
for ds_name, info in results.items():
    print(f"Dataset: {ds_name}  (file used: {info['file']})  samples: {len(info['samples'])}")
    display(info['df'])

for name, loader in loaders.items():
    print(f"{name}: {len(loader)} samples")

Found datasets: ['008-zeek-mixed', '009-zeek-malicious', '010-zeek-mixed', '011-zeek-mixed', '012-zeek-mixed', '013-zeek-mixed', '014-zeek-malicious', '015-zeek-malicious']
Dataset: 008-zeek-mixed  (file used: ../dataset-private/008-zeek-mixed/data/conn.log.labeled)  samples: 3


,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,...,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,label,detailedlabel
0,1655087149.561088,CqgEIg2ZNCcPtcGxR3,192.241.219.163,42578,147.32.81.167,8443,tcp,None,0.000008,0,...,True,0,S,2,80,0,0,(empty),Label.MALICIOUS,External-Attack-to-Device
1,1655114971.933980,CixIe92eFrkGR5bO33,147.32.81.167,63542,20.42.73.25,443,tcp,ssl,0.595694,13426,...,False,10840,ShDdcAaCFf,20,11880,16,5318,(empty),Label.BENIGN,Microsoft
2,1655156720.373058,CHMhR32Qw0EY5kowi3,218.75.31.110,9912,147.32.81.167,25691,tcp,None,0.000006,0,...,True,0,S,2,120,0,0,(empty),Label.MALICIOUS,External-Attack-to-Device


Dataset: 009-zeek-malicious  (file used: ../dataset-private/009-zeek-malicious/data/conn.log.labeled)  samples: 3


,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,...,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,label,detailedlabel
0,12376.445278,Cvi4BI2adPWyRLBg71,192.168.1.113,58992,115.2.225.225,443,tcp,None,0.298144,0.0,...,False,0,Sr,1,52,1,40,(empty),Label.MALICIOUS,(empty)
1,22641.434225,Cksnxw3lAoD4dxJYw3,192.168.1.113,64389,189.90.217.247,80,tcp,None,None,NaN,...,False,0,F,1,40,0,0,(empty),Label.MALICIOUS,(empty)
2,26352.309850,CjHGMg2BCyGtl8Sefa,192.168.1.113,56108,181.29.34.146,443,tcp,None,None,NaN,...,False,0,S,1,52,0,0,(empty),Label.MALICIOUS,(empty)


Dataset: 010-zeek-mixed  (file used: ../dataset-private/010-zeek-mixed/data/conn.log.labeled)  samples: 3


...

Dataset: 014-zeek-malicious  (file used: ../dataset-private/014-zeek-malicious/data/conn.log.labeled)  samples: 3


,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,...,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,label,detailedlabel
0,21434.852411,Coq0rW1fjX7EghNtpb,fd2d:ab8c:225:0:81d3:fd3e:7f62:aec8,65122,fd2d:ab8c:225::1,53,udp,dns,0.000383,34.0,...,False,0,Dd,1,82,1,142,(empty),Label.MALICIOUS,(empty)
1,76372.259197,CavbrN3JafJ34CR11j,fd2d:ab8c:225:0:81d3:fd3e:7f62:aec8,52783,fd2d:ab8c:225::1,53,udp,dns,0.000553,34.0,...,False,0,Dd,1,82,1,197,(empty),Label.MALICIOUS,(empty)
2,84680.304520,CA9Dfc2kRpn2SG6se1,fe80::f106:5909:32ae:f4cb,131,ff02::1:ffcc:1f02,130,icmp,None,None,NaN,...,False,0,None,1,72,0,0,(empty),Label.BENIGN,Multicast


Dataset: 015-zeek-malicious  (file used: ../dataset-private/015-zeek-malicious/data/conn.log.labeled)  samples: 3


,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,...,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,label,detailedlabel
0,95143.245416,CuGEWc1t8OTCunCJK6,192.168.1.113,52509,14.175.137.205,80,tcp,None,None,None,...,False,0,S,1,52,0,0,(empty),Label.MALICIOUS,(empty)
1,117186.178567,CXXqKo23Dgm09tR489,192.168.1.113,49425,167.46.197.204,443,tcp,None,None,None,...,False,0,S,1,52,0,0,(empty),Label.MALICIOUS,(empty)
2,140738.481957,CE6och4AjC0TSsM5H9,192.168.1.113,49523,172.18.5.190,80,tcp,None,None,None,...,False,0,S,1,52,0,0,(empty),Label.MALICIOUS,(empty)


008-zeek-mixed: 5603 samples
009-zeek-malicious: 79401 samples
010-zeek-mixed: 5310 samples
011-zeek-mixed: 8722 samples
012-zeek-mixed: 3810 samples
013-zeek-mixed: 26241 samples
014-zeek-malicious: 26738 samples
015-zeek-malicious: 197722 samples


## Pipeline

In [ ]:
#feature processing
feature_extraction = FeatureExtraction()

#preprocessing
scaler = StandardScaler() 
preprocessor = PreprocessingWrapper()
preprocessor.add_step("scaler", scaler)
#other steps? pca?

#classifier
model = SGDClassifier(loss='hinge', penalty='l2') 
classifier = SKLearnClassifierWrapper(model)

# reporting
logger = Logger(experiment_name = experiment_name)


In [ ]:

# datasets we have from before - select one (or a list) to train on!
# main loop
    # call batch from dataset
    # process features
    # preprocessing (scaling)
    # train on model with validation, logger for metrics!
    # save model after the whole dataset is done
    # reporting, metrics, plots, etc.
dataset_to_train_on = "008"
loader = loaders[dataset_to_train_on]

for i in loader.batches():
    batch = loader.next_batch()
    X, y = feature_extraction.process_batch(batch)
    sum_labeled_flows = len(y)

    X_train, X_val, y_gt_train, y_gt_val = train_test_split(X, y, test_size=validation_split, random_state=42)
    
    preprocessor.partial_fit(X_train)
    X_train_processed = preprocessor.transform(X_train)
    y_pred_train = classifier.partial_fit(X_train_processed, y)

    # predict on validation set
    X_val_processed = preprocessor.transform(X_val)
    y_pred_val = classifier.predict(X_val_processed)
    logger.save_training_results(
         y_pred_train, y_gt_train, y_pred_val, y_gt_val, sum_labeled_flows
    )

In [ ]:
# final test?
# on unrelated dataset?